In [18]:
import os
import re
import json
import operator
from datetime import datetime, timedelta
from apiclient import discovery
from apiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd
import googletrans
from googletrans import Translator

In [6]:
# Init
api_service_name = "youtube"
api_version = "v3"
API_key = ""
# List of your watch-history files (allow several accounts)
files = ['watch-history-kids.json']
service = build("youtube", "v3",developerKey=API_key)

In [7]:
#list video_IDs
video_IDs=[]

In [8]:
#json.load(open(files[0], encoding="utf8"))

In [9]:
#json.load(open(files[0], encoding="utf8"))[0]['titleUrl']

In [10]:
#spl_word = 'watch?v='
#json.load(open(files[0], encoding="utf8"))[0]['titleUrl'].partition(spl_word)[2] 

In [11]:
for f in files:
    for x in json.load(open(f, encoding="utf8")):
        spl_word = 'watch?v='
        try:
            video_ID = x['titleUrl'].partition(spl_word)[2]
            video_IDs.append(video_ID)
        except:
            pass

In [12]:
len(video_IDs)

138

In [13]:
video_IDs[0]

'ODugYNZdGUM'

In [14]:
title=[ ]
channel_title=[]
desc=[]
date=[]
tags=[]
liked=[ ]
disliked=[ ]
views=[ ]
comment=[ ]

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey = API_key)

for i in range(len(video_IDs)):
    request = youtube.videos().list(part="snippet",id=video_IDs[i])
    snippetdata = request.execute()
    req = youtube.videos().list(part="statistics",id=video_IDs[i])
    statistics = req.execute()
    try:
        title.append(snippetdata['items'][0]['snippet'].get('title'))
        channel_title.append(snippetdata['items'][0]['snippet'].get('channelTitle'))
        desc.append(snippetdata['items'][0]['snippet'].get('description'))
        date.append(snippetdata['items'][0]['snippet'].get('publishedAt'))
        tags.append(snippetdata['items'][0]['snippet'].get('tags')) 
        liked.append(statistics['items'][0]['statistics'].get('likeCount'))
        disliked.append(statistics['items'][0]['statistics'].get('dislikeCount'))
        views.append(statistics['items'][0]['statistics'].get('viewCount'))
        comment.append(statistics['items'][0]['statistics'].get('commentCount'))
    except IndexError:
        pass

In [15]:
i

137

In [19]:
data={'title':title,'channel_title':channel_title,'desc':desc,'date':date,'tags':tags,'liked':liked,'disliked':disliked,'views':views,'comment':comment}
df=pd.DataFrame(data)
df.head(5)

,title,channel_title,desc,date,tags,liked,disliked,views,comment
0,Deema in Police Story Adventure,sisters fun tube 2,Deema plays in a Police Story about looking fo...,2020-03-30T15:00:18Z,"[police, police story, treasure, funny video, ...",2665,825,2233928,None
1,Emma Pretend Play Babysitting Cry Baby Dolls w...,Toys and Colors,Emma pretend play babysitting her cry baby dol...,2018-08-16T12:00:02Z,"[emma, pretend play, emma pretend play, baby d...",323060,154770,151747806,None
2,TÔ MÀU TRANH CÁT CÔNG CHÚA TRÁI TIM - Colored ...,Chim Xinh Channel,Chị Chim Xinh chơi TÔ MÀU TRANH CÁT CÔNG CHÚA ...,2019-10-19T11:05:00Z,"[tranh cát công chúa, tranh cát, tô màu tranh ...",None,None,6301067,0
3,Jannie Pretend Play Dress Up with Beautiful Dr...,Toys and Colors,Jannie pretend play dress up with beautiful dr...,2019-06-12T12:00:11Z,"[pretend play, jannie, toys and colors, dress ...",147739,73251,50060040,None
4,Jobs Career & Professions Song | Wendy & Frien...,Toys and Colors,"Wendy, Alex, and Liam pretend play sing to the...",2019-06-10T12:00:02Z,"[toys and colors, nursery rhymes, children son...",668541,339595,216688073,None


In [25]:
#translate to english
translator = Translator(service_urls=['translate.googleapis.com'])

In [26]:
df['title_en'] = df['title'].apply(translator.translate,  dest='en').apply(getattr, args=('text',))
df.head(5)

AttributeError: 'NoneType' object has no attribute 'group'